# Wine Price Prediction - Model Deployment


## Data and Use case

[**Wine Reviews** - 130k wine reviews with variety, location, winery, price, and description](https://www.kaggle.com/zynicide/wine-reviews/home)

This dataset is available on Kaggle and contains around 130k of wine reviews. The data was scraped from [WineEnthusiast](http://www.winemag.com/?s=&drink_type=wine) on November 22nd, 2017.

I plan to use this dataset to develop a model that predicts wine price for specified set of parameters, like wine variety, region, desired quality. Such model, may be integrated into an application that runs on a mobile device to suggest price range during wine shopping without need to do online search.

## Deploy our Keras model in WML

First, install the Python API client from pypi.

In [1]:
# !rm -rf $PIP_BUILD/watson-machine-learning-client

In [2]:
#Install the WML client if required.
# !pip install watson-machine-learning-client --upgrade
# !pip install wget

Authenticate to the WML service on IBM Cloud.

In [3]:
# The code was removed by Watson Studio for sharing.

In [6]:
wml_credentials={
  "url": service_url,
  "username": user,
  "password": password,
  "instance_id": instance_id
}

Instantiate WML client.

In [7]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient

In [8]:
wml_client = WatsonMachineLearningAPIClient(wml_credentials)

Details about the WML service instance are available in service_instance.details.

In [9]:
# List information about your WML service instance.
print(wml_client.service_instance.details)

{'metadata': {'created_at': '2018-11-27T10:31:45.197Z', 'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/c9424f5f-0323-4aa3-87e1-eae64d00164b', 'modified_at': '2018-11-27T10:31:45.197Z', 'guid': 'c9424f5f-0323-4aa3-87e1-eae64d00164b'}, 'entity': {'region': 'us-south', 'account': {'id': '73bc3224d4573659240aaeec4bfcf74c', 'type': 'PAYG', 'name': "Oleg Kholod's Account"}, 'tags': None, 'deployments': {'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/c9424f5f-0323-4aa3-87e1-eae64d00164b/deployments'}, 'source': 'Bluemix', 'status': 'Active', 'plan_id': '3f6acf43-ede8-413a-ac69-f8af3bb0cbfe', 'published_models': {'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/c9424f5f-0323-4aa3-87e1-eae64d00164b/published_models'}, 'organization_guid': 'N/A', 'plan': 'lite', 'space_guid': 'N/A', 'usage': {'capacity_units': {'current': 0, 'limit': 180000000}, 'gpu_count_v100': {'current': 0, 'limit': 0}, 'expiration_date': '2019-04-01T00:00:00.000Z', 'model_count': {'current'

In [10]:
# List information about the stored models.
wml_client.repository.list_models()

------------------------------------  ---------------------  ------------------------  --------------
GUID                                  NAME                   CREATED                   FRAMEWORK
5a9d0fa6-2541-4b43-8785-43fd7959c8ba  Wine Price Prediction  2018-12-01T10:53:13.122Z  tensorflow-1.5
------------------------------------  ---------------------  ------------------------  --------------


In [11]:
# List the deployed models.
wml_client.deployments.list()

------------------------------------  ---------------------  ------  --------------  ------------------------  --------------  -------------
GUID                                  NAME                   TYPE    STATE           CREATED                   FRAMEWORK       ARTIFACT TYPE
5cac7bd6-a611-427d-8c18-240bb85287fb  Wine Price Prediction  online  DEPLOY_SUCCESS  2018-12-01T10:53:24.484Z  tensorflow-1.5  model
------------------------------------  ---------------------  ------  --------------  ------------------------  --------------  -------------


Let's prepare archive with our model for upload.

In [18]:
!tar -cvzf wine-model.tgz wine-price-prediction.model.keras.final.fitted.h5

wine-price-prediction.model.keras.final.fitted.h5


In [19]:
!ls *.tgz

wine-model.tgz


Store the archive file in the WML service repository, then list all the files stored in the repository.

In [20]:
# Save the downloaded file to the WML repository.
props = {wml_client.repository.ModelMetaNames.NAME: 'Wine Price Prediction',
              wml_client.repository.ModelMetaNames.AUTHOR_NAME: "Oleg Kholod", 
              wml_client.repository.ModelMetaNames.FRAMEWORK_NAME: "tensorflow", 
              wml_client.repository.ModelMetaNames.FRAMEWORK_VERSION: "1.5"}

model_details = wml_client.repository.store_model('wine-model.tgz', props)
wml_client.repository.list_models()

------------------------------------  ---------------------  ------------------------  --------------
GUID                                  NAME                   CREATED                   FRAMEWORK
5a9d0fa6-2541-4b43-8785-43fd7959c8ba  Wine Price Prediction  2018-12-01T10:53:13.122Z  tensorflow-1.5
------------------------------------  ---------------------  ------------------------  --------------


In [21]:
# Create the online deployment.
model_uid = wml_client.repository.get_model_uid(model_details)
model_deployment = wml_client.deployments.create(model_uid, name = 'Wine Price Prediction')

wml_client.deployments.list()



#######################################################################################

Synchronous deployment creation for uid: '5a9d0fa6-2541-4b43-8785-43fd7959c8ba' started

#######################################################################################


INITIALIZING
DEPLOY_IN_PROGRESS.
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='5cac7bd6-a611-427d-8c18-240bb85287fb'
------------------------------------------------------------------------------------------------


------------------------------------  ---------------------  ------  --------------  ------------------------  --------------  -------------
GUID                                  NAME                   TYPE    STATE           CREATED                   FRAMEWORK       ARTIFACT TYPE
5cac7bd6-a611-427d-8c18-240bb85287fb  Wine Price Prediction  online  DEPLOY_SUCCESS  2018-12-01T10:53:24.

In [22]:
# Verify the online scoring endpoint.
model_scoring_url = wml_client.deployments.get_scoring_url(model_deployment)

print(model_scoring_url)

https://us-south.ml.cloud.ibm.com/v3/wml_instances/c9424f5f-0323-4aa3-87e1-eae64d00164b/deployments/5cac7bd6-a611-427d-8c18-240bb85287fb/online


## Check the deployed model

Let's check the prediction by the model deployed to WML and the same model running in Watson Studio. I use a dummy data in both cases, we just need to see the same result.

In [24]:
import numpy as np

x = np.zeros((1146,), dtype=float)

# Score data and predict price.
scoring_data = {'fields': ['inputs'], 'values': [np.array(x).tolist()]}
predictions = wml_client.deployments.score(model_scoring_url, scoring_data) 

print(predictions)

{'fields': ['prediction', 'prediction_classes', 'probability'], 'values': [[[1.970050573348999], [1], [1.970050573348999]]]}


Run the model in Watson Studio.

In [25]:
from keras.models import load_model
model = load_model('wine-price-prediction.model.keras.final.fitted.h5')

In [27]:
model.predict(np.zeros((1,1146), dtype=float))

array([[ 1.97005057]], dtype=float32)

We can see that both calls produced the same result.